## 数据准备

In [3]:
import numpy as np
# 状态设定
state = np.array(['认真复习', '简单复习', '没有复习'])
grade = np.array(['A+', 'A', 'A-', 'B+', 'B', 'B-', 'C+', 'C', 'C-'])
# 初始概率分布
pi = np.array([1/3, 1/3, 1/3])
# 转移矩阵
t = np.array([
    [0.4, 0.3, 0.3],
    [0.3, 0.4, 0.3],
    [0.3, 0.3, 0.4]
])
# 发射矩阵
e = np.zeros([3,9])
e[0, :9]=1/9    # 把第0行所有列的元素设为1/9
e[1, 3:9]=1/6   # 把第1行3-9列的元素设为1/6
e[2, 5:9]=1/4   # 把第2行5-9列的元素设为1/4
# 让打印时,只打印小数点后两位
np.set_printoptions(precision=2, suppress=True)
# 打印结果
print("初始概率矩阵：\n",pi)
print("转移矩阵：\n",t)
print("发射矩阵：\n",e)

初始概率矩阵：
 [0.33 0.33 0.33]
转移矩阵：
 [[0.4 0.3 0.3]
 [0.3 0.4 0.3]
 [0.3 0.3 0.4]]
发射矩阵：
 [[0.11 0.11 0.11 0.11 0.11 0.11 0.11 0.11 0.11]
 [0.   0.   0.   0.17 0.17 0.17 0.17 0.17 0.17]
 [0.   0.   0.   0.   0.   0.25 0.25 0.25 0.25]]


## hmmlearn

In [ ]:
! pip install hmmlearn

In [15]:
from hmmlearn.hmm import CategoricalHMM
# 创建一个分类HMM模型
# n_components=3: 设置3个隐藏状态（认真复习、简单复习、没有复习）
hmm = CategoricalHMM(n_components=3,random_state=42)
hmm.startprob_ = pi  # startprob=pi: 设置初始概率分布
hmm.transmat_ = t  # transmat=t: 设置状态转移矩阵
hmm.emissionprob_ = e # emissionprob=e: 设置发射概率矩阵（各状态下观测值的概率分布）
hmm.n_features = 9  #设置观测值的数量（9个等级：A+到C-）

In [16]:
# 创建样本数据，表示学生的成绩序列 例如：0代表A+，4代表B，6代表C+
data = np.array([0, 4, 2, 6, 1])
# hmmlearn要求输入数据为二维数组，其中第一维是时间序列，第二维是特征, axis=1表示在第二个维度上扩展
data = np.expand_dims(data, axis=1)
data


array([[0],
       [4],
       [2],
       [6],
       [1]])

In [ ]:
# 预测最可能的状态序列
states = hmm.predict(data)
states

array([0, 0, 0, 2, 0])

In [ ]:
# 计算给定观测序列的似然概率 : hmm.score()返回的是对数似然概率，需要用exp转换成实际概率
print(f"观测序列的似然概率为: {np.exp(hmm.score(data))}")
# 可以看到结果很小,因为观测值的可能性很多,所以每一种可能性的概率都很小

观测序列的似然概率为: 8.284786081615825e-07


也可以使用HMM生成符合我们之前约束条件的数据

In [20]:
datas , states = hmm.sample(10000)

In [21]:
t_2 = np.zeros([3,3])
for i in range(3):
    current = np.where(states == i)[0]
    next_index = current+1
    next_index = next_index[:-1]

    tmp = states[next_index]
    for j in range(3):
        t_2[i][j] = np.where(tmp==j)[0].shape[0]/np.shape(tmp)[0]
print(t_2)

[[0.41 0.29 0.3 ]
 [0.31 0.4  0.29]
 [0.29 0.3  0.41]]


In [22]:
e_2 = np.zeros([3,9])
for i in range(3):
    current = np.where(states == i)[0]
    next_index = current+1
    next_index = next_index[:-1]
    tmp = datas[current]
    for j in range(9):
        e_2[i][j] = np.where(tmp==j)[0].shape[0]/np.shape(tmp)[0]
print(e_2)

[[0.12 0.11 0.11 0.12 0.11 0.11 0.1  0.11 0.11]
 [0.   0.   0.   0.16 0.16 0.17 0.17 0.17 0.18]
 [0.   0.   0.   0.   0.   0.25 0.25 0.24 0.25]]
